C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Clienti per area territoriale\Clienti indiretta\Agenzie

In [2]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime

from zebra import remov_duplicates, human_format
from babel.numbers import format_number

In [3]:
idx = pd.IndexSlice

In [27]:
pd.set_option('display.max_rows', 100)

# Sources

## Parametriche

In [4]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [5]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod["Codice PRODOTTO"].apply(lambda x: str(x))
cod_prod = cod_prod.loc[:, ['MDM', 'GTM BDG', 'Solution','Sub solution',"Codice PRODOTTO"]]

sales_director = par["Sales_director"]
sales_director = sales_director.loc[:, ["Sales Director", "Area territoriale", "Rete"]]
sales_director["Sales Director"] = sales_director["Sales Director"].str.lower()

agente_ordine = par["Agente_ordine"]

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

li_prodotti = par["Linea_prodotti"]

## Parco clienti da Licenze e abbonamenti

In [6]:
clienti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Diretta Indiretta IndirettaWKInfo\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Diretta Indiretta IndirettaWKInfo") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Diretta Indiretta IndirettaWKInfo", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(clienti+file, na_values="#", dtype={"Data inizio contratt":object, "Data Inizio Abb.":object, "Data Fine Abb.":object, "Cliente FATTURA":str, "Codice ABBONAMENTO":str})
    data["Tipo Licenza"] = file.split("_")[1].split(".")[0]
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(clienti+file))
    dfs.append(data)

cli = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in cli.columns:
    if "Data" in col:
        
        cli[col] = cli[col].apply(lambda x: (float(x) - 25569) * 86400)
        cli[col] = pd.to_datetime(cli[col], unit="s")
    else:
        pass

cli.rename(columns={'Unnamed: 10':'Descrizione ABBONAMENTO'}, inplace=True )

## All accounts da Salesforce

In [7]:
accounts_sf = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_all_accounts.csv", encoding='latin-1', sep=";")

# Lista di clienti con determinati prodotti

## Filtri

In [8]:
#Inserire valore per stato abbonamento e per stato contratto

stato_abb = (cli["Stato Esteso"].str.contains("ATTIVO")) & (cli["Stato Sintetico Contratto"].isin(["Attivo","Sospeso"]))

#Inserire colonne da considerare

columns = ["Cliente MERCE","Segmento Aggregato Cl.Merce", 'Codice agente',"Agente dell'ORDINE", 'RSM Agente', "Area territoriale",'Solution', 'Sub solution', "Sistema","Fornitura (SW)","Valore PV New"]

#Inserire area territoriale

area_territ = ['Rete IPSOA', 'Rete WKI']


## Creazione dataframe filtrato

In [9]:
clienti = cli.loc[stato_abb, :].copy()

clienti["Sales Director"] = clienti["Sales Director"].str.lower()
clienti["Codice ABBONAMENTO"] = clienti.loc[:,"Codice ABBONAMENTO"].apply(str)

def cli_prod_saldir_ag(df_):
    df = clienti.merge(cod_prod, how="left", left_on="Codice ABBONAMENTO", right_on="Codice PRODOTTO")\
          .merge(sales_director, how="left", on="Sales Director")\
          .merge(agente_ordine, how="left", left_on="Agente dell'Abbonamento", right_on="Codice agente")
    df["Cliente MERCE"] = df["Cliente MERCE"].apply(str)
    
    return df.loc[df["Area territoriale"].isin(area_territ), columns]

clienti = clienti.pipe(cli_prod_saldir_ag)

## Valore PV per cliente

In [10]:
clienti["PV Cliente"] = clienti.groupby("Cliente MERCE", dropna=False)["Valore PV New"].transform(sum)

## Modifica valore Solution per comprendere uno o più valori della Sub Solution

In [11]:
clienti.loc[(clienti["Sub solution"] == "Tuttotel FE"),  "Solution"] = "Tuttotel FE"
clienti.loc[(clienti["Sub solution"] == "DIGY EC"),  "Solution"] = "DIGY EC"

In [12]:
solutions = ["Sistema Professionista", "B.Point", "B.Point Paghe", "Giotto", "Arca Evolution", "Genya Bilancio", "Tuttomodelli", "Tuttotel", "Tuttotel FE", "DIGY EC", "B.Point Azienda", "Prima Nota"]

clienti = clienti.loc[clienti["Solution"].isin( solutions),:].copy()

In [13]:
clienti.loc[~clienti["Solution"].isin(["Sistema Professionista", "B.Point"]), "Fornitura (SW)"] = ""

In [14]:
tabellone = clienti.groupby(["RSM Agente", "Agente dell'ORDINE", "Cliente MERCE", "PV Cliente", "Solution", "Fornitura (SW)"], dropna=False).agg({"Valore PV New":"sum"}).unstack([4,5]).reset_index(col_level=1).droplevel(level=0, axis=1)

In [15]:
tabellone.columns = tabellone.columns.map('|'.join).str.strip('|')

## Aggiunta della ragione sociale

In [16]:
tabellone["Codice SAP"] = tabellone["Cliente MERCE"].astype(str).apply(lambda x: "IT-" + x.zfill(10))
tabellone = tabellone.merge(accounts_sf, how="left", left_on="Codice SAP", right_on="WK Account Number")

## Ultima definizione colonne

In [18]:
tabellone = tabellone.loc[:,['RSM Agente', "Agente dell'ORDINE", "Account Name", "Cliente MERCE", 'PV Cliente',
                             'Sistema Professionista|ON PREMISE',  'B.Point|ON PREMISE',
                             'Sistema Professionista|SAAS', 'B.Point|SAAS', 
                             'B.Point Paghe', 'Giotto',
                             'Arca Evolution', 'Genya Bilancio',
                             'Tuttomodelli',
                             'Tuttotel', 'Tuttotel FE',
                             'DIGY EC',
                             'B.Point Azienda', 
                             'Prima Nota']].copy()
tabellone.rename(columns={"Agente dell'ORDINE":"Agenzia", "RSM Agente":"ICC"}, inplace=True)

In [20]:
for rsm in tabellone["ICC"].unique():
    
    with pd.ExcelWriter(f"C:/Users/Raffaele.Sportiello/OneDrive - Wolters Kluwer/Documents/Analisi clienti/Clienti per area territoriale/Clienti indiretta/Agenzie/ICC/{rsm}.xlsx") as writer:
        
        for elemento in tabellone.loc[tabellone["ICC"] == rsm, "Agenzia"].unique():
            
            tabellone.loc[(tabellone["ICC"] == rsm) & (tabellone["Agenzia"] == elemento),
                         ["Account Name", "Cliente MERCE", 'PV Cliente',
                         'Sistema Professionista|ON PREMISE',  'B.Point|ON PREMISE',
                         'Sistema Professionista|SAAS', 'B.Point|SAAS', 
                         'B.Point Paghe', 'Giotto',
                        'Arca Evolution', 'Genya Bilancio',
                         'Tuttomodelli',
                         'Tuttotel', 'Tuttotel FE',
                         'DIGY EC',
                         'B.Point Azienda',
                          'Prima Nota']].to_excel(writer, sheet_name=elemento, index=False, float_format = "%0.0f")


In [88]:
clienti.groupby(["Cliente MERCE", "Agente dell'ORDINE", "Solution"], dropna=False).agg({"Valore PV New":"sum", "PV Cliente":"max"}).loc["1158832"]

Valore PV New  PV Cliente
Agente dell'ORDINE   Solution                              
GAM SOLUZIONI S.R.L. B.Point            2470.73      7339.5
                     Tuttotel            791.16      7339.5
                     Tuttotel FE         220.34      7339.5

In [91]:
cli.groupby(["Cliente MERCE", "Agente dell'Abbonamento", "Prodotto (SW)"], dropna=False).agg({"Valore PV New":"sum"}).loc[1158832]

Valore PV New
Agente dell'Abbonamento Prodotto (SW)                     
J075                    B.POINT                    2470.73
                        BASILEA                     257.91
                        SIAR                        463.24
                        SUITE BILANCIO              779.80
                        TANDEM                      257.90
                        TUTTOTEL                   1011.50
                        VALUTAZIONE AZIENDA         193.43
                        WEBDESK                    1904.99